In [76]:
import folium
import pandas as pd
from folium.plugins import HeatMap

In [77]:
df = pd.read_csv('raw_data.csv', sep=";")

In [78]:
df.columns = ["photo_id", "user_internal_id", "user_name", "lat", "longt", "date_taken", "date_upload",  "visit"]

In [79]:
# Converter a coluna date_taken para o tipo datetime
df['date_taken'] = pd.to_datetime(df['date_taken'])

# Agrupar os dados por user_name e calcular a diferença em dias entre a primeira e a última foto
df['first_date_taken'] = df.groupby('user_name')['date_taken'].transform('min')
df['last_date_taken'] = df.groupby('user_name')['date_taken'].transform('max')
df['days_diff'] = (df['last_date_taken'] - df['first_date_taken']).dt.days

# Marcar os usuários como "Turistas" ou "Local" com base na diferença em dias
df['tipo_usuario'] = 'Indefinido'
df.loc[df['days_diff'] <= 40, 'tipo_usuario'] = 'Turista'
df.loc[df['days_diff'] > 40, 'tipo_usuario'] = 'Local'

# Descartar colunas auxiliares adicionadas
df.drop(['first_date_taken', 'last_date_taken', 'days_diff'], axis=1, inplace=True)

# # Escrever os dados manipulados de volta para o arquivo CSV
# df.to_csv('seu_arquivo_manipulado.csv', index=False)


In [81]:
mapa = folium.Map(location=[-7.144126, -34.857480], zoom_start=12)

# Iterar sobre as linhas do DataFrame
for index, row in df.iterrows():
    # Adicionar um ponto para cada foto no mapa
    cor = 'blue' if row['tipo_usuario'] == 'Local' else "red"
  
    folium.Circle(location=[row['lat'], row['longt']], radius=0.00001, fill=True, color=cor).add_to(mapa)


coordenadas_turistas = df[df['tipo_usuario'] == 'Turista'][['lat', 'longt']].values.tolist()
coordenadas_locais = df[df['tipo_usuario'] == 'Local'][['lat', 'longt']].values.tolist()

# Adicionar camadas de densidade de pontos para turistas e locais
heatmap_turistas = HeatMap(coordenadas_turistas, radius=15).add_to(mapa)
heatmap_locais = HeatMap(coordenadas_locais, radius=15).add_to(mapa)

# Salvar o mapa como um arquivo HTML
mapa.save('mapa_fotos.html')